<a href="https://colab.research.google.com/github/21JN1A4218/FMML-MODULUS---IIIT-HYBERABAD/blob/main/Module_01_Lab_02_MLPractice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine learning terms and metrics

FMML Module 1, Lab 2<br>


 In this lab, we will show a part of the ML pipeline by extracting features, training and testing

In [1]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets
# set randomseed
rng = np.random.default_rng(seed=42)

In this lab, we will use the California Housing dataset. There are 20640 samples, each with 8 attributes like income of the block, age of the houses per district etc. The task is to predict the cost of the houses per district.

Let us download and examine the dataset.

In [2]:
 dataset =  datasets.fetch_california_housing()
 # print(dataset.DESCR)  # uncomment this if you want to know more about this dataset
 # print(dataset.keys())  # if you want to know what else is there in this dataset
 dataset.target = dataset.target.astype(np.int) # so that we can classify
 print(dataset.data.shape)
 print(dataset.target.shape)

(20640, 8)
(20640,)


<ipython-input-2-60ae2e9a125e>:4: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dataset.target = dataset.target.astype(np.int) # so that we can classify


Here is a function for calculating the 1-nearest neighbours

In [3]:
def NN1(traindata, trainlabel, query):
  diff  = traindata - query  # find the difference between features. Numpy automatically takes care of the size here
  sq = diff*diff # square the differences
  dist = sq.sum(1) # add up the squares
  label = trainlabel[np.argmin(dist)] # our predicted label is the label of the training data which has the least distance from the query
  return label

def NN(traindata, trainlabel, testdata):
  # we will run nearest neighbour for each sample in the test data
  # and collect the predicted classes in an array using list comprehension
  predlabel = np.array([NN1(traindata, trainlabel, i) for i in testdata])
  return predlabel

We will also define a 'random classifier', which randomly allots labels to each sample

In [4]:
def RandomClassifier(traindata, trainlabel, testdata):
  # in reality, we don't need these arguments

  classes = np.unique(trainlabel)
  rints = rng.integers(low=0, high=len(classes), size=len(testdata))
  predlabel = classes[rints]
  return predlabel

Let us define a metric 'Accuracy' to see how good our learning algorithm is. Accuracy is the ratio of the number of correctly classified samples to the total number of samples. The higher the accuracy, the better the algorithm.

In [5]:
def Accuracy(gtlabel, predlabel):
  assert len(gtlabel)==len(predlabel), "Length of the groundtruth labels and predicted labels should be the same"
  correct = (gtlabel==predlabel).sum() # count the number of times the groundtruth label is equal to the predicted label.
  return correct/len(gtlabel)

Let us make a function to split the dataset with the desired probability.

In [6]:
def split(data, label, percent):
  # generate a random number for each sample
  rnd = rng.random(len(label))
  split1 = rnd<percent
  split2 = rnd>=percent
  split1data = data[split1,:]
  split1label = label[split1]
  split2data = data[split2,:]
  split2label = label[split2]
  return split1data, split1label, split2data, split2label

We will reserve 20% of our dataset as the test set. We will not change this portion throughout our experiments

In [7]:
testdata, testlabel, alltraindata, alltrainlabel = split(dataset.data, dataset.target, 20/100)
print('Number of test samples = ', len(testlabel))
print('Number of other samples = ', len(alltrainlabel))
print('Percent of test data = ', len(testlabel)*100/len(dataset.target),'%')

Number of test samples =  4144
Number of other samples =  16496
Percent of test data =  20.07751937984496 %


## Experiments with splits

Let us reserve some of our train data as a validation set

In [8]:
traindata, trainlabel, valdata, vallabel = split(alltraindata, alltrainlabel, 75/100)

What is the accuracy of our classifiers on the train dataset?

In [9]:
trainpred = NN(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Train accuracy using nearest neighbour is ", trainAccuracy)

trainpred = RandomClassifier(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Train accuracy using random classifier is ", trainAccuracy)

Train accuracy using nearest neighbour is  1.0
Train accuracy using random classifier is  0.164375808538163


For nearest neighbour, the train accuracy is always 1. The accuracy of the random classifier is close to 1/(number of classes) which is 0.1666 in our case.

Let us predict the labels for our validation set and get the accuracy

In [10]:
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour is ", valAccuracy)

valpred = RandomClassifier(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using random classifier is ", valAccuracy)

Validation accuracy using nearest neighbour is  0.34108527131782945
Validation accuracy using random classifier is  0.1688468992248062


Validation accuracy of nearest neighbour is considerably less than its train accuracy while the validation accuracy of random classifier is the same. However, the validation accuracy of nearest neighbour is twice that of the random classifier.

Now let us try another random split and check the validation accuracy

In [11]:
traindata, trainlabel, valdata, vallabel = split(alltraindata, alltrainlabel, 75/100)
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy of nearest neighbour is ", valAccuracy)

Validation accuracy of nearest neighbour is  0.34048257372654156


You can run the above cell multiple times to try with different random splits.
We notice that the accuracy is different for each run, but close together.

Now let us compare it with the accuracy we get on the test dataset.

In [12]:
testpred = NN(alltraindata, alltrainlabel, testdata)
testAccuracy = Accuracy(testlabel, testpred)
print('Test accuracy is ', testAccuracy)

Test accuracy is  0.34917953667953666


### Try it out for yourself and answer:
1. How is the accuracy of the validation set affected if we increase the percentage of validation set? What happens when we reduce it?
2. How does the size of the train and validation set affect how well we can predict the accuracy on the test set using the validation set?
3. What do you think is a good percentage to reserve for the validation set so that thest two factors are balanced?

Answer for both nearest neighbour and random classifier. You can note down the values for your experiments and plot a graph using  <a href=https://matplotlib.org/stable/gallery/lines_bars_and_markers/step_demo.html#sphx-glr-gallery-lines-bars-and-markers-step-demo-py>plt.plot<href>. Check also for extreme values for splits, like 99.9% or 0.1%

**How is the accuracy of the validation set affected if we increase the percentage of validation set? What happens when we reduce it?**
Increasing the Percentage of Validation Set:

Pros:

A larger validation set can provide a more reliable estimate of the model's performance because it evaluates the model on a larger and more representative sample of the data.
It helps reduce the risk of overfitting because the model has fewer training samples to fit the training data, making it less likely to memorize noise in the data.
Cons:

With a larger validation set, you have fewer samples available for training the model. This can lead to slower training times and potentially underfitting, where the model fails to capture the underlying patterns in the data due to limited training data.
It may not be suitable for cases where data is limited, as you need to strike a balance between having enough data for training and enough for validation.
Effect on Accuracy:

Initially, as you increase the validation set percentage, validation accuracy tends to be more reliable and may increase. However, beyond a certain point, further increases in the validation set percentage can lead to a decrease in training set size and may result in reduced training performance.
Reducing the Percentage of Validation Set:

Pros:

With a smaller validation set, you have more data available for training the model. This can lead to faster training times and potentially better modeling of the underlying patterns in the data.
Cons:

A smaller validation set may lead to less reliable estimates of the model's performance because it evaluates the model on a smaller and potentially less representative sample of the data.
There's a higher risk of overfitting because the model has more training data and may memorize noise in the training data.
Effect on Accuracy:

Initially, as you reduce the validation set percentage, training accuracy may improve due to having more data for training. However, validation accuracy may become less reliable and may decrease because of the reduced sample size for validation.

**How does the size of the train and validation set affect how well we can predict the accuracy on the test set using the validation set?**

The size of the training and validation sets can indeed affect how well you can predict the accuracy on the test set using the validation set. Here's how:

Larger Training Set:

Pros:

With a larger training set, your model has more data to learn from. This can help the model generalize better to the underlying patterns in the data, which can lead to improved predictive performance on both the validation and test sets.
A model trained on a larger dataset is less likely to overfit because it has more representative examples to learn from.
Cons:

If the validation set is relatively small compared to the training set, it may not provide a reliable estimate of how well the model will perform on unseen data (i.e., the test set). The validation set may not capture the full diversity of data that the test set contains.
Larger Validation Set:

Pros:

A larger validation set provides a more stable estimate of your model's performance. It reduces the variance in your performance estimates because you're evaluating the model on a more representative sample of the data.
Cons:

With a larger validation set, you have fewer data points available for training the model. This can potentially lead to slower training times and may limit the model's capacity to capture complex patterns in the data.
Smaller Training Set:

Pros:

A smaller training set may allow you to train your model faster because you're using fewer data points. This can be advantageous in scenarios where training time is a critical factor.
Cons:

If the training set is too small, the model may not generalize well to the validation and test sets. It may overfit to the training data, leading to poor predictive performance on unseen data.
Smaller Validation Set:

Pros:

With a smaller validation set, you have more data points available for training the model, which can be beneficial for model performance.
Cons:

A smaller validation set may provide less reliable estimates of model performance. The performance scores on the validation set may vary more due to the smaller sample size.

**What do you think is a good percentage to reserve for the validation set so that thest two factors are balanced?**
Common Split Ratios:

A common split ratio is to reserve 70-80% of your data for training and the remaining 20-30% for validation. This is often a good starting point for many datasets and machine learning tasks.
Another common split is the 60-20-20 split, where you allocate 60% for training, 20% for validation, and 20% for testing.
Small Datasets:

If you have a small dataset (e.g., a few hundred samples), you might consider using a larger proportion for validation (e.g., 30% or more) to ensure that you have a sufficiently large validation set for reliable performance estimation.
Large Datasets:

With very large datasets (e.g., millions of samples), you can often afford to allocate a smaller percentage for validation (e.g., 10-20%) because you still have a substantial number of validation samples.
Complex Models:

If you're training a highly complex model with a large number of parameters (e.g., deep neural networks), it can be beneficial to have a larger validation set to get more reliable estimates of model performance. In such cases, you might lean toward the higher end of the validation set percentage range.
Limited Data:

When you have a very limited amount of data, consider techniques like k-fold cross-validation. In k-fold cross-validation, the data is divided into k subsets (folds), and the model is trained and validated k times, each time using a different fold as the validation set. This helps you make efficient use of your limited data.
Domain Knowledge:

Domain knowledge can also play a role in deciding the split ratio. If you have prior knowledge about the problem, the data, or the expected model performance, it can guide your choice.

## Multiple Splits

One way to get more accurate estimates for the test accuracy is by using <b>crossvalidation</b>. Here, we will try a simple version, where we do multiple train/val splits and take the average of validation accuracies as the test accuracy estimation. Here is a function for doing this. Note that this function will take a long time to execute.

In [13]:
# you can use this function for random classifier also
def AverageAccuracy(alldata, alllabel, splitpercent, iterations, classifier=NN):
  accuracy = 0
  for ii in range(iterations):
    traindata, trainlabel, valdata, vallabel = split(alldata, alllabel, splitpercent)
    valpred = classifier(traindata, trainlabel, valdata)
    accuracy += Accuracy(vallabel, valpred)
  return accuracy/iterations # average of all accuracies

In [14]:
print('Average validation accuracy is ', AverageAccuracy(alltraindata, alltrainlabel, 75/100, 10, classifier=NN))
testpred = NN(alltraindata, alltrainlabel, testdata)
print('test accuracy is ',Accuracy(testlabel, testpred) )

Average validation accuracy is  0.33584635395170215
test accuracy is  0.34917953667953666


This is a very simple way of doing cross-validation. There are many well-known algorithms for cross-validation, like k-fold cross-validation, leave-one-out etc. This will be covered in detail in a later module. For more information about cross-validation, check <a href=https://en.wikipedia.org/wiki/Cross-validation_(statistics)>Cross-validatioin (Wikipedia)</a>

### Questions
1. Does averaging the validation accuracy across multiple splits give more consistent results?
2. Does it give more accurate estimate of test accuracy?
3. What is the effect of the number of iterations on the estimate? Do we get a better estimate with higher iterations?
4. Consider the results you got for the previous questions. Can we deal with a very small train dataset or validation dataset by increasing the iterations?


**Does averaging the validation accuracy across multiple splits give more consistent results?**
Reduced Variability: When you use a single fixed validation set, the choice of that particular subset of data can have a significant impact on the reported validation accuracy. By averaging the results across multiple splits, you reduce the influence of any one random split, leading to more stable and representative estimates of your model's performance.

Better Generalization: Averaging validation accuracy across multiple splits allows your model to be evaluated on different subsets of the data during each iteration. This promotes better generalization, as the model learns to perform well on various portions of the dataset, which can be more indicative of its true performance on unseen data.

Robustness: It makes your evaluation more robust to the initial random split. A single unfortunate split could lead to overly optimistic or pessimistic estimates of model performance, which can be misleading. Averaging over multiple splits smooths out this variability.

More Information: With k-fold cross-validation, you obtain k different estimates of model performance. This not only gives you a more reliable measure of your model's performance but also provides insights into the variability of that performance.

**Does it give more accurate estimate of test accuracy?**
Reduced Bias: By using multiple validation sets and averaging their performance, you reduce the potential bias introduced by a single validation set. A single fixed validation set might accidentally contain data that is either easier or harder to classify, leading to an inaccurate estimate of your model's generalization performance. Averaging over multiple validation sets helps mitigate this bias.

Better Generalization: Cross-validation or repeated sampling exposes your model to various subsets of the data during each iteration. This encourages your model to learn more robust and generalizable patterns from the data, as it has to perform well on different slices of the dataset. Consequently, the reported performance is more indicative of how your model is likely to perform on new, unseen data, such as the test set.

Robustness: Averaging over multiple splits makes your performance evaluation more robust to outliers or anomalies that might be present in a single validation set. An unfortunate random split of data in a single validation set can lead to overly optimistic or pessimistic performance estimates, which can be misleading.

While cross-validation and repeated random sampling can provide a more accurate estimate of your model's generalization performance, it's important to keep in mind that they are still estimates. The actual performance on the test set can still vary due to differences in the test data distribution, data collection issues, or other factors not present in your training and validation data.

**What is the effect of the number of iterations on the estimate? Do we get a better estimate with higher iterations?**
Fewer Iterations:

With a small number of iterations (e.g., 2 or 3), you may get a less stable estimate of model performance. The results can vary significantly depending on the specific data splits.
There's a higher risk that the performance estimate might be overly optimistic or pessimistic due to the limited number of evaluations.
Moderate Iterations:

Increasing the number of iterations to a moderate level (e.g., 5 to 10) generally leads to more stable and reliable estimates of model performance.
With more iterations, the variability in performance estimates is reduced, and you get a better sense of the model's average performance.
Diminishing Returns:

Beyond a certain point, increasing the number of iterations may not significantly improve the accuracy of the estimate. The law of diminishing returns applies, meaning that the incremental gain in estimate quality becomes smaller as you add more iterations.
Moreover, a higher number of iterations requires more computational resources and time, which may not always be practical.
Computational Cost:

Keep in mind that each additional iteration adds computational cost. If you have limited computational resources or time constraints, you might need to strike a balance between the number of iterations and available resources.

**Consider the results you got for the previous questions. Can we deal with a very small train dataset or validation dataset by increasing the iterations?**
Small Training Dataset:

If you have a very small training dataset, increasing the number of iterations can be beneficial to some extent because it allows your model to be trained on different subsets of the limited training data during each iteration.
However, a very small training dataset may still lead to issues like overfitting, where the model struggles to generalize to new data. More iterations can help in estimating how well the model might perform on different subsets of the data, but it can't magically generate more information from a small dataset.
Small Validation Dataset:

When you have a small validation dataset, increasing the number of iterations can provide more stable and reliable estimates of model performance compared to using a single small validation set. This reduces the risk of your performance estimate being overly influenced by a specific random split.
While increasing iterations can improve the stability of the estimate, it doesn't change the fact that you have limited data available for model evaluation. The overall performance estimate may still be less precise than desired.
Limitations:

There are limitations to how much increasing the number of iterations can compensate for a very small dataset. If the dataset is extremely small, it may be challenging to obtain accurate and reliable estimates of model performance, no matter how many iterations you use.
Additionally, computational resources and time constraints may limit your ability to use an extremely high number of iterations.
In cases where you have a very small training dataset or validation dataset, it's important to be aware of the limitations and potential issues related to data scarcity. You should consider strategies to address this limitation, such as:

Data Augmentation: If possible, apply data augmentation techniques to artificially increase the effective size of your training dataset.
Transfer Learning: Explore the use of pre-trained models and transfer learning, which can leverage knowledge learned from larger datasets.
Feature Engineering: Focus on feature engineering and domain knowledge to extract more information from the existing data.
Collect More Data: If feasible, consider collecting additional data to increase the size of your dataset, which can lead to more robust model training and evaluation.